## Phase: Full ML Pipeline V2

Let's now say we want to train *both* V1 and V2 models in parallell. The typical reason for this is we are not clear on if V2 is actually better and want to do A/B test, or V2 has different use (i.e. different customent segment).

Whatever the reasoning, we want to train both stages of pipeline for V1 and V2 independently, and Linea + Airflow allows to do this easily!

In [1]:
# NBVAL_IGNORE_OUTPUT
import lineapy

#### Linea API allows to easily see previously stored artifacts:

In [2]:
# NBVAL_IGNORE_OUTPUT
lineapy.catalog()

cleaned_data_housing:2022-04-07T11:03:59 created on 2022-04-07 11:03:59.558632
cleaned_data_housing:2022-04-07T11:06:08 created on 2022-04-07 11:06:08.697765
linea_model_housing_V2:2022-04-07T11:08:06 created on 2022-04-07 11:08:06.498065
linea_model_housing_V2:2022-04-07T11:10:48 created on 2022-04-07 11:10:48.945714
housing_data_advanced_features:2022-04-07T11:12:46 created on 2022-04-07 11:12:46.849067
housing_data_advanced_features:2022-04-07T11:14:55 created on 2022-04-07 11:14:55.617774
cleaned_data_housing:2022-04-07T11:17:13 created on 2022-04-07 11:17:13.809893
linea_model_housing:2022-04-07T11:17:41 created on 2022-04-07 11:17:41.231510
lot_area_by_neighborhood:2022-04-07T11:18:37 created on 2022-04-07 11:18:37.532364
sale_price_by_neighborhood:2022-04-07T11:18:37 created on 2022-04-07 11:18:37.548730
sale_price_by_year_neighborhood:2022-04-07T11:18:37 created on 2022-04-07 11:18:37.559762

#### Now we can send both to Airflow a a single pipeline:

In [3]:
# NBVAL_IGNORE_OUTPUT
preprocessing_art = lineapy.get("cleaned_data_housing")
modeling_art = lineapy.get("linea_model_housing")

In [4]:
# NBVAL_IGNORE_OUTPUT
preprocessing_art_V2 = lineapy.get("housing_data_advanced_features")
modeling_art_V2 = lineapy.get("linea_model_housing_V2")

In [5]:
# NBVAL_SKIP
import os
lineapy.to_pipeline(
    [
        preprocessing_art.name, modeling_art.name,
        preprocessing_art_V2.name, modeling_art_V2.name
    ], 
    framework='AIRFLOW', 
    pipeline_name="data_housing_pipeline",
    dependencies=[
        ('data_housing_pipeline_cleaned_data_housing','data_housing_pipeline_linea_model_housing'),
        ('data_housing_pipeline_housing_data_advanced_features','data_housing_pipeline_linea_model_housing_V2')
    ],
    output_dir = os.environ.get("AIRFLOW_HOME","~/airflow")+"/dags")


Pipeline source generated in the directory: /Users/simba/airflow/dags

Generated python module data_housing_pipeline.py

Added Airflow DAG named data_housing_pipeline_dag. Start a run from the Airflow UI or CLI.

Generated Dockerfile data_housing_pipeline_Dockerfile

Generated requirements file data_housing_pipeline_requirements.txt

PosixPath('/Users/simba/airflow/dags')